# Appariement SPlink sur données de décès

## Environnement 

In [1]:
### Installation des packages splink et recordlinkage
!pip install splink
!pip install recordlinkage

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 7.7 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.2/516.2 kB 6.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 7.0 MB/s eta 0:00:0000:0100:01
  Created wheel for phonetics: filename=phonetics-1.0.5-py2.py3-none-any.whl size=8695 sha256=e14086151f161979349ac0dc1f606cab1ea1aadc07981be03f1b40d2e7ba81bd
  Stored in directory: /home/onyxia/.cache/pip/wheels/40/63/73/d2c3bed2dc6df7c34ec9e1402761bed3519a4c0d152858b190
Successfully built phonetics
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 16.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00a 0:00:01


In [2]:
### Import des librairies nécessaires
import pandas as pd
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.blocking_rule_library import block_on

# Pour S3
import os
import s3fs

## Chargement des données

Deux tables avec les mêmes individus (même nombre de lignes et mêmes indentifiants). La table **deces_perturb** a été dégradée (ajout volontaire d'imprécisions dans les différentes colonnes identifiantes).

In [3]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "projet-ssplab"

# Import de la table des décès
FILE_KEY_S3 = "appariements/deces.parquet"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    deces = pd.read_parquet(file_in)

# Import de la table des décès perturbée
FILE_KEY_S3 = "appariements/deces_perturb.parquet"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    deces_perturb = pd.read_parquet(file_in)

Besoin de passer les colonnes de noms/prénoms en minuscules dans la table de gauche.

In [4]:
deces['nom_etat_civil'] = deces['nom_etat_civil'].str.lower()
deces['prenoms_etat_civil'] = deces['prenoms_etat_civil'].str.lower()

## Appariement 

Nombre de lignes à sélectionner dans les deux bases (sur les 26 millions)

In [5]:
nb_lignes = 900000

Les individus ont les mêmes identifiants ligne à ligne (la base perturbée contient les mêmes individus, triés dans le même ordre)

In [6]:
df_gauche = deces.iloc[:nb_lignes]
df_droite = deces_perturb.iloc[:nb_lignes]

In [7]:
df_gauche['ident_deces'].equals(df_droite['ident_deces'])

True

Part de lignes ayant subi une "perturbation" lors de la création de la table de gauche

In [8]:
df_droite.agg(part=('perturbation', 'sum')) / len(df_droite)

,perturbation
part,0.182558


In [9]:
df_gauche = df_gauche.drop(['datenaiss', 'datedeces', 'lieudeces', 'adeces'], axis=1)

Initialisation de l'objet Linker

In [10]:
linker = DuckDBLinker([df_gauche, df_droite], {"link_type": "link_only", "unique_id_column_name": "ident_deces"})

Règle de blocage

In [25]:
blocking_rules = [
        "l.lieunaiss = r.lieunaiss and (substr(l.nom_etat_civil, 1, 3) = substr(r.nom_etat_civil, 1, 3) or substr(l.nom_etat_civil, length(l.nom_etat_civil) -2 , 3) = substr(r.nom_etat_civil, length(r.nom_etat_civil) - 2, 3))"
    ]


In [22]:
#count = linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)
#count

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.Chart(...)

In [26]:
print("Nombre de paires conservées " 
f"{linker.count_num_comparisons_from_blocking_rule(' or '.join(blocking_rules))}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Nombre de paires conservées 18428262


### Règle de comparaison des champs

In [27]:
comparisons_list = [
        cl.jaro_winkler_at_thresholds("nom_etat_civil", [0.95, 0.88], term_frequency_adjustments = True),
        cl.jaro_winkler_at_thresholds("prenoms_etat_civil", [0.95, 0.88], term_frequency_adjustments = True),
        cl.exact_match("mnais_etat_civil", term_frequency_adjustments=True),
        cl.exact_match("jnais_etat_civil", term_frequency_adjustments=True)
    ]

### Définition du dictionnaire des paramètres

In [29]:
linkage_settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": comparisons_list,
    "unique_id_column_name": "ident_deces"
}

## Estimation des paramètres

Ici, cas particulier où on sait qu'à une ligne de droite correspond exactement une ligne de gauche.
Il faud fixer la probabilité **probability_two_random_records_match** à nb_lignes^2

Nombre de paires à utiliser pour réaliser les estimations des paramètres u et m. 
Conseil de la documentation : "au moins 10 millions, mais 1 milliard pour les grosses tables".

Choix des variables `nom_etat_civil` et `prenoms_etat_civil` pour s'approcher du Get Started de SPlink... L'idée serait de bloquer sur des paires de probables vrais matches pour estimer la proportion d'erreurs/imprécisions dans les données. Besoin de comprendre davantage la documentation pour faire mieux.

Pour estimer le paramètre **m**, on pourrait utiliser des données labélisées. Ceci pourrait faciliter l'estimation.
Voici la commande de la documentation `linker.estimate_m_from_label_column("social_security_number")`

In [35]:
nb_paires_estimation = 1e7

In [36]:
%%time
linker = DuckDBLinker([df_gauche, df_droite], linkage_settings)
linker.estimate_u_using_random_sampling(max_pairs = nb_paires_estimation)
session_nom = linker.estimate_parameters_using_expectation_maximisation(block_on("nom_etat_civil"))
session_prenom = linker.estimate_parameters_using_expectation_maximisation(block_on("prenoms_etat_civil"))

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - nom_etat_civil (no m values are trained).
    - prenoms_etat_civil (no m values are trained).
    - mnais_etat_civil (no m values are trained).
    - jnais_etat_civil (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."nom_etat_civil" = r."nom_etat_civil"

Parameter estimates will be made for the following comparison(s):
    - prenoms_etat_civil
    - mnais_etat_civil
    - jnais_etat_civil

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - nom_etat_civil


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Iteration 17: Largest change in params was 0.000137 in the m_probability of jnais_etat_civil, level `Exact match`


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Iteration 18: Largest change in params was -9.63e-05 in the m_probability of jnais_etat_civil, level `All other comparisons`

EM converged after 18 iterations

Your model is fully trained. All comparisons have at least one estimate for their m and u values


CPU times: user 1h 22min 3s, sys: 1min 22s, total: 1h 23min 25s
Wall time: 3min


Autre stratégie (ne fonctionne pas a priori) : 

In [ ]:
#%%time
#
#linker = DuckDBLinker([df_gauche, df_droite], linkage_settings)
#linker.estimate_u_using_random_sampling(max_pairs = nb_paires_estimation)
#
#training_blocking_rule_nom_prenom = block_on(["nom_etat_civil", "prenoms_etat_civil"])
#training_session_nom_prenom = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule_nom_prenom)
#
#training_blocking_rule_anais = block_on("anais_etat_civil")
#training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule_anais)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - nom_etat_civil (no m values are trained).
    - prenoms_etat_civil (no m values are trained).
    - mnais_etat_civil (no m values are trained).
    - jnais_etat_civil (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."nom_etat_civil" = r."nom_etat_civil") AND (l."prenoms_etat_civil" = r."prenoms_etat_civil")

Parameter estimates will be made for the following comparison(s):
    - mnais_etat_civil
    - jnais_etat_civil

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - nom_etat_civil
    - prenoms_etat_civil

Iteration 1: Largest change in params was 0.0914 in the m_probability of mnais_etat_civil, level `All other comparisons`
Iteration 2: Largest change in params was -0.0357 in probability_two_random_records_match
Iteratio

### Analyse du modèle 

In [ ]:
linker.match_weights_chart() 

In [ ]:
linker.m_u_parameters_chart()

In [ ]:
linker.unlinkables_chart()

### Classification des paires

Attention à vérifier l'impact du seuil **0.5**

In [ ]:
#results = linker.predict(threshold_match_probability=0.5)
results = linker.predict()
results_pandas = results.as_pandas_dataframe()
results_pandas.shape


### Résolution des conflits

In [ ]:
sql = f"""
with ranked as

(
select *,
row_number() OVER (
    PARTITION BY ident_deces_l order by match_weight desc
    ) as row_number
from {results.physical_name}
)

select *
from ranked
where row_number = 1


"""
results = linker.query_sql(sql)

In [ ]:
results

### Evaluation de la qualité

In [ ]:
def compute_performance_metrics_FEBRL(results, dataset_size):
    """
    Compute performance metrics of a record linkage process on FEBRL synthetic data.
    The assumption is that the size of the two datasets is the same and every record 
    from dataset A has exactly one match in dataset B.

            Parameters:
                    results (pandas DataFrame): Output from the linkage process
                    dataset_size (int): Length of both datasets to be linked

            Returns:
                    performance_metrics (tuple): Tuple of metrics (TP, TN, FP, FN, precision, recall, F-measure)
    """
    results['actual'] = (results['ident_deces_l'].str.extract(r'(Deces_2021_\d+)') 
                                == results['ident_deces_r'].str.extract(r'(Deces_2021_\d+)'))
    TP = sum(results['actual'])
    FP = sum(~results['actual'])
    #Pairs that were removed in the indexing phase must be taken into account to compute True and False negatives
    FN = dataset_size - TP
    TN = dataset_size*dataset_size - TP - FN - FP

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    Fscore = 2 * precision * recall / (precision + recall)
    performance_metrics = (TP, TN, FP, FN, precision, recall, Fscore)
    return(performance_metrics)

def print_performance_metrics(linkage_output, dataset_size):
    """
    Prints performance metrics of a record linkage process on synthetic data.
    The assumption is that the size of the two datasets is the same and every record 
    from dataset A has exactly one match in dataset B.

            Parameters:
                    results (pandas DataFrame): Output from the linkage process
                    dataset_size (int): Length of both datasets to be linked

            Returns:
                    None
    """
    TP, TN, FP, FN, precision, recall, Fscore = compute_performance_metrics_FEBRL(results, dataset_size)
    print(f"Vrais positifs : {TP:,}".replace(',', ' '))
    print(f"Vrais négatifs : {TN:,}".replace(',', ' '))
    print(f"Faux positifs : {FP:,}".replace(',', ' '))
    print(f"Faux négatifs : {FN:,}".replace(',', ' '))
    print(f"Précision : {precision:.4}")
    print(f"Rappel : {recall:.4}")
    print(f"F-mesure : {Fscore:.4}")

print_performance_metrics(results, nb_lignes)

